<!-- background: #fff4e3-->
<!-- color: #2d2422 -->
<div align="center">
    <img src="./images/ocetrac_logo_transparent.png" width=400/>
    <h3><span style="font:tradegothic; color:gray">morphological image processing for monitoring ocean temperature extremes</span></h3>
    <hr>
    <h2><span style="font:futura; color:purple">Hillary Scannell<sup>1</sup></span><span style="font:futura; style=color:black">, Luanne Thompson<sup>2</sup>, Daniel Whitt<sup>3</sup>, David John Gagne<sup>4</sup>, Ryan Abernathey<sup>1</sup> & Julius Busecke<sup>1</sup></span></h2>
    <h6><sup>1</sup>Lamont-Doherty Earth Observatory, Columbia University, Palisades, NY, USA</h6>
    <h6><sup>2</sup>School of Oceanography, University of Washington, Seattle, WA, USA</h6>
    <h6><sup>3</sup>NASA Ames Research Center, Mountain View, CA, USA</h6>
    <h6><sup>4</sup>National Center for Atmospheric Research, Boulder, CO, USA</h6>
</div>


<!-- background: #fff4e3-->
<!-- color: #2d2422 -->

<div align="center">
    <h3>Abstract</h3>
</div>
Dangerous hot-water events, called marine heatwaves, cause prolonged periods of thermal stress in the marine environment that can lead to widespread coral bleaching, harmful algal blooms, unproductive fisheries, and even economic loss. Anticipating the paths of destructive marine heatwaves remains a challenge owing to the complex spatiotemporal evolution of these events. We present a novel open source package called Ocetrac that implements morphological image processing and tracking to aid in the analysis of marine heatwave detection and movement. We discuss how this approach can be applied to monitor other extreme ocean conditions, such as deoxygenation and acidification.


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import xarray as xr
import scipy.ndimage
from skimage.measure import regionprops 
from skimage.measure import label as label_np
import dask.array as dsa

In [ ]:
endpoint_url = 'https://ncsa.osn.xsede.org'
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': endpoint_url},)

path = "Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True)
print(ds)

In [ ]:
# Define a mask
mask = ds.sst.isel(time=0, zlev=0).fillna(-999)
mask = mask.where(mask==-999, other = 1)
mask = mask.where(mask==1, other = 0)

plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
mask.plot(transform=ccrs.PlateCarree(), vmin=0, vmax=1, cmap='Greys_r', extend='max', add_colorbar=False, label=False)
ax.set_title('')
ax.background_patch.set_visible(False)